# Introduction to Neural Net Classifiers
This tries to classify iris flower types based on four measurements. See https://archive.ics.uci.edu/ml/datasets/Iris/ for details

In [ ]:
using Flux
using MLDatasets
using Random
using Statistics
using IterTools: ncycle 
using ProgressMeter
using Plots

Type `y` when prompted the first time.

In [ ]:
labels = MLDatasets.Iris.labels();
features = MLDatasets.Iris.features();

In [ ]:
labels

In [ ]:
unique(labels)

In [ ]:
features

In [ ]:
normed_features = Flux.normalise(features, dims=2); # normalize the data
klasses = sort(unique(labels));
onehot_labels = Flux.onehotbatch(labels, klasses); # identify the classes


In [ ]:
normed_features

## Split into Training/Testing Data

In [ ]:
n = size(normed_features)[2]
Random.seed!(100);
idx = shuffle(1:n);
split = 0.7; # 70% for training and the balance for testing is common
train_idx = idx[1:floor(Int, split *n)];
test_idx = idx[floor(Int, split *n)+1:end];

n_train = length(train_idx);
n_test = length(test_idx);

X_train = normed_features[:, train_idx]
y_train = onehot_labels[:, train_idx]

X_test = normed_features[:, test_idx]
y_test = onehot_labels[:, test_idx]

# arrange as (x,y) pairs for training
train_data = Flux.Data.DataLoader((X_train, y_train),batchsize=n_train)


## Set up Neural net

In [ ]:
Random.seed!(200);
# Shallow network with no hidden layers
# model = Chain(Dense(4, 100, sigmoid),Dense(100,3));
model = Dense(4,3);
p = params(model);
p0 = deepcopy(p);

## Train

In [ ]:
loss = (x, y)-> Flux.Losses.logitcrossentropy(model(x), y); # set the loss function
opt = ADAM(0.1); # pick the classifier and the ''learning rate''

In [ ]:
numEpochs = 10000;

losses=Float32[]
ProgressMeter.ijulia_behavior(:clear);
pmeter = Progress(length(train_data)*numEpochs)

cb() = begin
    l=loss(X_train,y_train)
    push!(losses, l)
    next!(pmeter; showvalues=[(:loss,l)])
end 

Flux.train!(loss, p, ncycle(train_data, numEpochs), opt, cb=cb)

In [ ]:
plot(1:length(losses),losses,legend=false, xscale=:log10)
xlabel!("Epoch")
ylabel!("Training Data Loss")

## Assess Accuracy

In [ ]:
println("Training Error")
mean(Flux.onecold(model(X_train)) .== Flux.onecold(y_train))

In [ ]:
println("Testing Error")

mean(Flux.onecold(model(X_test)) .== Flux.onecold(y_test))